In [1]:
# pip install scikit-learn


In [32]:
%load_ext autoreload
%autoreload
import pandas as pd
from processor.process import *
from processor.preprocess import *
from sklearn.ensemble import RandomForestClassifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df_raw = pd.read_csv("/Users/jatintyagi/Downloads/train-3.csv")

In [4]:
target_variable= "Failure"

In [5]:
df_raw[target_variable]

0       No
1       No
2       No
3       No
4       No
        ..
7900    No
7901    No
7902    No
7903    No
7904    No
Name: Failure, Length: 7905, dtype: object

In [6]:
# def remove_lowcounts(df, rows):
#   val =round(rows*0.25)
#   t = []
#   for label,content in df.items():
#     if df[label].count() < val:
#       df.drop(columns = label, inplace=True)
#       t.append(label)
#   df.dropna(how='all', inplace=True)
#   return t

In [7]:
def plot_pdp(model, x, feat_name, clusters=None):
    #feat_name = feat_name or feat
    p = pdp.pdp_isolate(model, x, feature=feat_name, model_features=x.columns)
    return pdp.pdp_plot(p, feat_name, plot_lines=True,
                        cluster=clusters is not None,
                        n_cluster_centers=clusters)

In [8]:
def get_contributions(slice, model,df_trn ):
  if len(slice) < 10:
    sample = slice.copy()
  else:
    sample = get_sample(slice,round(len(slice)*.10))
  prediction, bias, contributions = ti.predict(model, sample)
  bias = bias[0]
  prediction = np.mean(prediction)
  result_list = []
  for i in range(len(sample)):
    idxs = np.argsort(contributions[i])
    for c, value, feature in sorted(zip(contributions[i][idxs], sample.iloc[i][idxs], df_trn.columns[idxs])):
      result = (feature, c, value)
      result_list.append(result)
  df_res = pd.DataFrame(result_list, columns=['feature','cont','value'])
  df_tree = df_res.groupby(['feature'])['cont'].agg('mean').reset_index()
  t_conts = df_tree.sort_values(by='cont',ascending=False)[:10]
  l_conts = df_tree.sort_values(by='cont',ascending=True)[:10]
  #Predictions = bias +
  return bias,prediction,df_tree,t_conts,l_conts,sample

In [9]:
# def display_visuals(df, df_trn, feats,imp,attr, target_variable, tmean):
#   if is_bool_dtype(df[feats]):
#     #would have been 1 hot encoded
#     bar_plot(df,feats, target_variable)
#     maxslice = imp
#     minslice = attr
#     return maxslice,minslice
#     display_texts(feats,imp,attr, target_variable, tmean)
#   elif is_numeric_dtype(df[feats]):
#     if df[feats].nunique() < 70:
#       display_texts(feats,imp,attr, target_variable,tmean)
#       bar_plot(df,feats, target_variable)
#     else:
#       display_texts(feats,imp,attr,target_variable,tmean)
#       scatter_plot(df,feats, target_variable)
#     df_cut,maxslice,minslice = qcut_df(df, df_trn, feats, target_variable, tmean)
#     #tmp = df_cut.groupby([feats])[target_variable].mean()
#     st.write(f'#### Bucketed "{feats}" across multiple categories Vs Average "{target_variable}"')
#     bar_ploty(df_cut, feats, target_variable)
#     #sn_bar_plot(df_cut)
#     #display_texts(feats,imp,attr)
#     return maxslice,minslice
#     #display_texts(feats,imp,attr)
#   elif df[feats].nunique() < 10:
#     #1-hot encoded
#     bar_plot(df,feats, target_variable)
#     minattr = df[feats].value_counts(ascending=True).index.tolist()[0]
#     maxslice,minslice = cat_slice(df_trn,feats,attr,minattr)
#     display_texts(feats,imp,attr,target_variable, tmean)
#     return maxslice,minslice
#   else:
#     #regular category
#     if df[feats].nunique() < 70:
#       bar_plot(df,feats, target_variable)
#     else:
#       scatter_plot(df,feats, target_variable)
#     minattr = df_trn[feats].value_counts(ascending=True).index.tolist()[0]
#     maxattr = df_trn[feats].value_counts(ascending=False).index.tolist()[0]
#     maxslice = df_trn[df_trn[feats]==maxattr]
#     minslice = df_trn[df_trn[feats]==minattr]
#     display_texts(feats,imp,attr,target_variable, tmean)
#     return maxslice,minslice

In [10]:
# pip install treeinterpreter pdpbox

In [11]:
rows = df_raw.shape[0]
cols = df_raw.shape[1]

In [12]:
printtmp1 = remove_lowcounts(df_raw, rows)


In [13]:
printtmp2 = remove_highcardinals_string(df_raw)
printtmp3 = remove_highcardinals_numeric(df_raw, target_variable)

printtmp2, printtmp3

(['Date'], ['ID'])

In [14]:
train_cats(df_raw)

In [15]:
get_date(df_raw)


In [16]:
df_trn, y_trn, nas = proc_df(df_raw, target_variable, max_n_cat=10)
trn_rows = df_trn.shape[0]
trn_cols = df_trn.shape[1]

basemodel = RandomForestClassifier(
    n_estimators=30, oob_score=True, n_jobs=-1)
basemodel.fit(df_trn, y_trn)


RandomForestClassifier(n_estimators=30, n_jobs=-1, oob_score=True)

In [17]:
result = get_score(basemodel, df_trn, y_trn)
p_rmse = result[0]
score = result[1]
oob = result[2]

In [18]:
basetopfeats = rf_imp_features(basemodel, df_trn)

        # basemodel,basetopfeats = feat_imp_check(basetopfeats)

basetopfeats

,features,imp
0,Temperature,0.294937
1,Humidity,0.216293
17,Hours Since Previous Failure,0.153807
15,Measure14,0.028302
7,Measure6,0.022587
11,Measure10,0.021388
9,Measure8,0.020999
12,Measure11,0.020586
10,Measure9,0.020334
16,Measure15,0.020207


In [19]:
n_estimators1 = [50, 100]
n_estimators2 = [50, 100]
# max_depth = [100, 200]
max_features = ['log2', 'sqrt']
min_samples_split = [3, 5]
min_samples_leaf = [2, 3]

# Commented out IPython magic to ensure Python compatibility.
features, model = get_features_class(score, oob, trn_cols, trn_rows, df_trn, y_trn, n_estimators1,
                                n_estimators2, max_features, min_samples_split, min_samples_leaf, basetopfeats, basemodel)

topfeat5 = features[:5]
topfeat5['Percent'] = topfeat5['imp']*100
topfeat5n = topfeat5[['features', 'Percent']]
topfeat5n.set_index('features', inplace=True)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END max_features=log2, min_samples_leaf=2, min_samples_split=3, n_estimators=50; total time=   0.5s
[CV] END max_features=log2, min_samples_leaf=2, min_samples_split=3, n_estimators=50; total time=   0.6s
[CV] END max_features=log2, min_samples_leaf=2, min_samples_split=3, n_estimators=50; total time=   0.6s
[CV] END max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=50; total time=   0.5s
[CV] END max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=50; total time=   0.6s
[CV] END max_features=log2, min_samples_leaf=2, min_samples_split=3, n_estimators=100; total time=   1.0s
[CV] END max_features=log2, min_samples_leaf=2, min_samples_split=3, n_estimators=100; total time=   1.0s
[CV] END max_features=log2, min_samples_leaf=2, min_samples_split=3, n_estimators=100; total time=   1.0s
[CV] END max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=50; total time

In [20]:
topfeat5n

,Percent
features,
Temperature,29.777690
Humidity,24.039776
Hours Since Previous Failure,19.740109
Measure9,1.739149
Measure5,1.688690


In [21]:
nfeatures = features.copy()

In [22]:
nfeatures

,features,imp
0,Temperature,0.297777
1,Humidity,0.240398
17,Hours Since Previous Failure,0.197401
10,Measure9,0.017391
6,Measure5,0.016887
9,Measure8,0.016634
15,Measure14,0.016259
14,Measure13,0.016243
8,Measure7,0.015549
7,Measure6,0.015533


In [23]:
nfeatures["features"].str.split("*")

0                      [Temperature]
1                         [Humidity]
17    [Hours Since Previous Failure]
10                        [Measure9]
6                         [Measure5]
9                         [Measure8]
15                       [Measure14]
14                       [Measure13]
8                         [Measure7]
7                         [Measure6]
11                       [Measure10]
2                         [Measure1]
5                         [Measure4]
12                       [Measure11]
19                     [?Date.month]
20              [?Date.day-of-month]
16                       [Measure15]
22                      [?Date.hour]
13                       [Measure12]
21               [?Date.day-of-week]
4                         [Measure3]
3                         [Measure2]
26             [Operator, Operator2]
28             [Operator, Operator4]
25             [Operator, Operator1]
31             [Operator, Operator7]
29             [Operator, Operator5]
2

In [24]:
new = nfeatures["features"].str.split("*", n=1, expand=True)
nfeatures["features"] = new[0]
if (new.shape[1] == 1):
    nfeatures['attribute'] = ""
else:
    nfeatures['attribute'] = new[1]

In [25]:
nfeatures



,features,imp,attribute
0,Temperature,0.297777,None
1,Humidity,0.240398,None
17,Hours Since Previous Failure,0.197401,None
10,Measure9,0.017391,None
6,Measure5,0.016887,None
9,Measure8,0.016634,None
15,Measure14,0.016259,None
14,Measure13,0.016243,None
8,Measure7,0.015549,None
7,Measure6,0.015533,None


In [37]:
feats1, imp1, attr1 = nfeatures['features'].iloc[0], (round(
            (nfeatures['imp'].iloc[0])*100, 2)), nfeatures['attribute'].iloc[0]
feats2, imp2, attr2 = nfeatures['features'].iloc[1], (round(
    (nfeatures['imp'].iloc[1])*100, 2)), nfeatures['attribute'].iloc[1]
feats3, imp3, attr3 = nfeatures['features'].iloc[2], (round(
    (nfeatures['imp'].iloc[2])*100, 2)), nfeatures['attribute'].iloc[2]
feats4, imp4, attr4 = nfeatures['features'].iloc[3], (round(
    (nfeatures['imp'].iloc[3])*100, 2)), nfeatures['attribute'].iloc[3]
feats5, imp5, attr5 = nfeatures['features'].iloc[4], (round(
    (nfeatures['imp'].iloc[4])*100, 2)), nfeatures['attribute'].iloc[4]

In [38]:
feats6, imp6, attr6 = nfeatures['features'].iloc[19], (round(
    (nfeatures['imp'].iloc[19])*100, 2)), nfeatures['attribute'].iloc[19]

In [39]:
feats6

'?Date.day-of-week'

In [29]:
%matplotlib inline

In [100]:
is_numeric_dtype(df_raw[feats5])

False

In [101]:
df_raw[feats5].nunique()

24

In [41]:
plot_cat_cat(df_raw,"Failure",feats6)

In [40]:
maxslice1, minslice1 = display_visuals(
                df_raw, df_trn, feats6, imp6, attr6, target_variable, tmean= df_raw[target_variable].count(), cls=True)

<Figure size 2016x720 with 0 Axes>

In [31]:
df_raw[feats1]

0       67
1       68
2       64
3       63
4       65
        ..
7900    60
7901    63
7902    60
7903    63
7904    67
Name: Temperature, Length: 7905, dtype: int64

In [103]:
ax = df_trn.plot.bar(x=df_trn.index, y=df_trn.values, figsize=(
        28, 10), color='mediumseagreen', rot=90)
ax.set_xlabel(feats1)
ax.set_ylabel(target_variable)

KeyError: 'None of [RangeIndex(start=0, stop=511, step=1)] are in the [columns]'